<a href="https://colab.research.google.com/github/arnisafazla/CS490/blob/main/CS_490_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/CS490/CS490-Senior-Research-Project

/content/drive/MyDrive/CS490/CS490-Senior-Research-Project


In [11]:
! git pull

Already up to date.


In [10]:
!git config --global user.email "arnisa.fazla@ug.bilkent.edu.tr"
!git config --global user.name "arnisafazla"

In [11]:
! git commit -m 'gradient penalty, tested with one class. implement projection next'

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
	modified:   dataset.py
	modified:   models/Base_WGAN.py
	modified:   models/__pycache__/Base_WGAN.cpython-37.pyc
	modified:   models/critic_models/LSTM_critic.py
	modified:   models/critic_models/__pycache__/LSTM_critic.cpython-37.pyc
	modified:   models/generator_models/__pycache__/LSTM_generator.cpython-37.pyc
	modified:   setup.ipynb
	modified:   tools.py

no changes added to commit


In [14]:
! git check-ignore -v __pycache__/

.gitignore:1:__pycache__	__pycache__/


In [ ]:
! git add setup.ipynb

In [ ]:
! echo 'models/__pycache__/' >> .gitignore

In [ ]:
! git add models/
! git add tools.py

In [10]:
! git push origin main

Everything up-to-date


In [ ]:
! ls -a

490_main.ipynb			  CS_490_main_WGAN.ipynb   models
conv_model.ipynb		  CS_490_notes.ipynb	   __pycache__
CS490_download_datasets.ipynb	  dataset.py		   PyMO
CS_490_main_generation_GAN.ipynb  download_datasets.ipynb  README.md
CS_490_main_generation.ipynb	  .git			   setup.ipynb
CS_490_main.ipynb		  .gitignore		   tools.py
CS_490_main_vanilla_GAN.ipynb	  .ipynb_checkpoints


In [3]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras.utils.vis_utils import plot_model

# path to the cloned repo
main_dir = '/content/drive/MyDrive/CS490/CS490-Senior-Research-Project'  
# path to the dataset folder
dataset_dir = '/content/drive/MyDrive/CS490/dataset'
%cd {main_dir}
sys.path.insert(0,main_dir)
from dataset import Dataset
from models.critic_models.LSTM_critic import define_critic
from models.generator_models.LSTM_generator import define_generator
from models.Base_WGAN import Base_WGAN

/content/drive/MyDrive/CS490/CS490-Senior-Research-Project


In [4]:
%cd ..
!mkdir generator_models
%cd generator_models
! touch LSTM_generator.py

/content/drive/MyDrive/CS490
/content/drive/MyDrive/CS490/generator_models


In [4]:
# path is the path to the dataset
# dataset = Dataset(["anger", "joy", "fear", "neutral", "sadness", "pride"], path=dataset_dir, step_size=10, frames=400)
dataset = Dataset(["anger"], path=dataset_dir, step_size=10, frames=400)

anger


100%|██████████| 65/65 [00:08<00:00,  7.26it/s]


In [5]:
Dataset.balance(dataset)

No. of samples in each class will be:  640


In [5]:
# save this as a separate file
config = {       # change hyperparams here
        'latent_dim' : 400,
        'in_shape' : (400, 69),
        'n_classes' : dataset.emotions.shape[0],
        'init_std' : 0.01,
        'critic_opt' : 'Adam', # RMS or Adam
        'critic_lr' : 0.005,
        'gen_opt' : 'Adam',
        'gen_lr' : 0.005,
        'wrong_labels' : False,    # feed in real data with wron labels as well - Remove?
        'epochs' : 20,
        'batch_size' : 128,
        'n_critic' : 3,
        'dataset_balanced' : True,
        'critic_batch_norm' : True,
        'critic_dropout' : 0,
        'gp_weight' : 10.0      # for the gradient penalty used in critic
      }

In [6]:
critic = define_critic(config)
# plot_model(critic, show_shapes=True, show_layer_names=True)
generator = define_generator(config)
# plot_model(generator, show_shapes=True, show_layer_names=True)

{'latent_dim': 400, 'in_shape': (400, 69), 'n_classes': 1, 'init_std': 0.01, 'critic_opt': 'Adam', 'critic_lr': 0.005, 'gen_opt': 'Adam', 'gen_lr': 0.005, 'wrong_labels': False, 'epochs': 20, 'batch_size': 128, 'n_critic': 3, 'dataset_balanced': True, 'critic_batch_norm': True, 'critic_dropout': 0, 'gp_weight': 10.0}


In [7]:
outputs_path = '/content/drive/MyDrive/CS490/models'
# the saved models, outputs, adn config files will go to this path
gan = Base_WGAN(outputs_path, 'WGAN', None, critic, generator, config, dataset)

In [8]:
generator_optimizer = keras.optimizers.Adam(
    learning_rate=0.0002, beta_1=0.5, beta_2=0.9
)
critic_optimizer = keras.optimizers.Adam(
    learning_rate=0.0002, beta_1=0.5, beta_2=0.9
)
# Define the loss functions for the critic,
# which should be (fake_loss - real_loss).
# We will add the gradient penalty later to this loss function.
def critic_loss(real, fake):
    real_loss = tf.reduce_mean(real)
    fake_loss = tf.reduce_mean(fake)
    return fake_loss - real_loss
# Define the loss functions for the generator.
def generator_loss(fake):
    return -tf.reduce_mean(fake)

# gan = Base_WGAN('/content/drive/MyDrive/CS490/models', 'WGAN', None, critic, generator, config, dataset)

# Compile the WGAN model.
gan.compile(
    c_optimizer=critic_optimizer,
    g_optimizer=generator_optimizer,
    g_loss_fn=generator_loss,
    c_loss_fn=critic_loss,
)

# Start training the model.
# wgan.fit(train_images, batch_size=BATCH_SIZE, epochs=epochs, callbacks=[cbk])

In [9]:
logs_path = '/content/drive/MyDrive/CS490/logs' # path to the tensorboard logs
gan.train(logs_path)

>1, 1/10, c_loss=9.921, g_loss=0.000
>1, 2/10, c_loss=9.865, g_loss=0.000
>1, 3/10, c_loss=9.779, g_loss=0.000
>1, 4/10, c_loss=9.549, g_loss=0.000
>1, 5/10, c_loss=9.051, g_loss=0.000
>1, 6/10, c_loss=8.166, g_loss=0.000
>1, 7/10, c_loss=7.268, g_loss=0.000
>1, 8/10, c_loss=5.811, g_loss=0.001
>1, 9/10, c_loss=4.747, g_loss=0.000
>1, 10/10, c_loss=3.069, g_loss=0.000
>2, 1/10, c_loss=2.025, g_loss=0.000
>2, 2/10, c_loss=1.347, g_loss=-0.000
>2, 3/10, c_loss=0.859, g_loss=-0.001
>2, 4/10, c_loss=0.591, g_loss=-0.001
>2, 5/10, c_loss=0.549, g_loss=-0.002
>2, 6/10, c_loss=0.741, g_loss=-0.003
>2, 7/10, c_loss=0.711, g_loss=-0.005
>2, 8/10, c_loss=0.531, g_loss=-0.008
>2, 9/10, c_loss=0.632, g_loss=-0.009
>2, 10/10, c_loss=0.485, g_loss=-0.013
>3, 1/10, c_loss=0.666, g_loss=-0.019
>3, 2/10, c_loss=0.436, g_loss=-0.024
>3, 3/10, c_loss=0.400, g_loss=-0.028
>3, 4/10, c_loss=0.520, g_loss=-0.033
>3, 5/10, c_loss=0.685, g_loss=-0.038
>3, 6/10, c_loss=0.446, g_loss=-0.038
>3, 7/10, c_loss=0.38

KeyboardInterrupt: ignored

In [10]:
[labels_real, X_real], y_real = dataset.generate_real_samples(1)

In [11]:
labels_real.numpy().item()

0.0

In [25]:
metrics = [list() for i in range(2)]
metrics[0].append(tf.convert_to_tensor(np.array([1,2,3])))
metrics[1].append(np.array([1,2,3]))

In [15]:
a = tf.convert_to_tensor(np.array([1,2,3]))

In [26]:
metrics[0] = metrics[0].numpy().tolist()
metrics[1] = metrics[1].tolist()

AttributeError: ignored

In [11]:

import json
with open(os.path.join('/content/drive/MyDrive/CS490/models/WGAN2022.01.26-13:43:55/epoch_1', 'train_metrics.txt'), 'w') as file:
  json.dump(metrics, file)

TypeError: ignored

In [10]:
for dir in os.listdir('/content/drive/MyDrive/CS490/models'):
  if dir[0:4] == 'WGAN':
    path = os.path.join('/content/drive/MyDrive/CS490/models', dir)
    ! rm -rf $path